# Data Original Feature

In [92]:
import os
import numpy as np
os.chdir("./Data/")

drug_size = 464
drug_fin, drug_tran, drug_fin_re = {}, {}, {}
def feature_extraction(data_name, drug_size):
    with open(data_name,'r') as r:
        col=0
        for line in r:
            line=line.strip('\n').split('\t')
            col=len(line)-1
            break
        drug_original_feature=np.zeros((drug_size,col))
        r.seek(0)
        for line in r:
            line=line.strip('\n').split('\t')
            name=line[0]
            if name in drug_fin:
                del line[0]
                drug_original_feature[drug_fin[name]]=np.array(line)
    return drug_original_feature
       
with open("drug_name.txt",'r') as r:
    index=0
    for line in r:
        line=line.strip('\n')
        drug_fin[line]=index
        drug_fin_re[index]=line
        index+=1
        
with open("drug_name_708.txt",'r') as r:
    index=0
    for line in r:
        line=line.strip('\n')
        if line in drug_fin:
            drug_tran[index]=drug_fin[line]
        index+=1

X_atc, X_struct, X_off, X_tar, X_drug_drug, X_drug_disease = None, None, None, None, None, None
X_atc = feature_extraction("Atc_ori.txt", drug_size)
X_struct = feature_extraction("Chemical_ori.txt", drug_size)
X_off = feature_extraction("Sider_ori.txt", drug_size)
X_tar = feature_extraction("Target_ori.txt", drug_size)
X_drug_drug = feature_extraction("Drug_drug_ori.txt", drug_size)
# with open("mat_drug_drug.txt",'r') as r:
#     col=0
#     for line in r:
#         line=line.strip('\n').split('\t')
#         col=len(line)-1
#         break
#     temp=np.zeros((drug_size,col))
#     r.seek(0)
#     for line in r:
#         line=line.strip('\n').split('\t')
#         name=line[0]
#         if name in drug_fin:
#             del line[0]
#             temp[drug_fin[name]]=np.array(line)
#     X_drug_drug=temp   
with open("mat_drug_disease.txt",'r') as r:
    col=0
    for line in r:
        line=line.strip('\n').split('\t')
        col=len(line)-1
        break
    temp=np.zeros((drug_size,col))
    r.seek(0)
    for line in r:
        line=line.strip('\n').split('\t')
        name=line[0]
        if name in drug_fin:
            del line[0]
            temp[drug_fin[name]]=np.array(line)
    X_drug_disease=temp   

FileNotFoundError: [Errno 2] No such file or directory: 'drug_name.txt'

# Data Similarity Feature

In [36]:
Sim_target = np.zeros((drug_size, drug_size))
Sim_sider = np.zeros((drug_size, drug_size))
Sim_atc = np.zeros((drug_size, drug_size))
Sim_struct = np.zeros((drug_size, drug_size))
Sim_drug_drug = np.zeros((drug_size, drug_size))

for index in range(drug_size):
    for index_2 in range(drug_size):
        if index_2 > index:  
            X_struct_index = (sum(X_struct[index])+sum(X_struct[index_2])-sum(abs(X_struct[index]-X_struct[index_2])))/2.0
            X_atc_index = (sum(X_atc[index])+sum(X_atc[index_2])-sum(abs(X_atc[index]-X_atc[index_2])))/2.0
            X_tar_index = (sum(X_tar[index])+sum(X_tar[index_2])-sum(abs(X_tar[index]-X_tar[index_2])))/2.0
            X_off_index = (sum(X_off[index])+sum(X_off[index_2])-sum(abs(X_off[index]-X_off[index_2])))/2.0
            X_drug_drug_index = (sum(Sim_drug_drug[index])+Sim_drug_drug(X_off[index_2])-sum(abs(Sim_drug_drug[index]-Sim_drug_drug[index_2])))/2.0

            if (sum(X_struct[index])+sum(X_struct[index_2])-X_struct_index)!=0:
                 Sim_struct[index][index_2]=X_struct_index/(sum(X_struct[index])+sum(X_struct[index_2])-X_struct_index)  
                 Sim_struct[index_2][index]=X_struct_index/(sum(X_struct[index])+sum(X_struct[index_2])-X_struct_index)  
            if (sum(X_atc[index])+sum(X_atc[index_2])-X_atc_index)!=0:
                    Sim_atc[index][index_2]=X_atc_index/(sum(X_atc[index])+sum(X_atc[index_2])-X_atc_index)  
                    Sim_atc[index_2][index]=X_atc_index/(sum(X_atc[index])+sum(X_atc[index_2])-X_atc_index)  
            if (sum(X_tar[index])+sum(X_tar[index_2])-X_tar_index)!=0:
                    Sim_target[index][index_2]=X_tar_index/(sum(X_tar[index])+sum(X_tar[index_2])-X_tar_index)
                    Sim_target[index_2][index]=X_tar_index/(sum(X_tar[index])+sum(X_tar[index_2])-X_tar_index)                   
            if (sum(X_off[index])+sum(X_off[index_2])-X_off_index)!=0:
                    Sim_sider[index][index_2]=X_off_index/(sum(X_off[index])+sum(X_off[index_2])-X_off_index)
                    Sim_sider[index_2][index]=X_off_index/(sum(X_off[index])+sum(X_off[index_2])-X_off_index)
            if (sum(X_drug_drug[index])+sum(X_drug_drug[index_2])-X_drug_drug_index)!=0:
                    Sim_drug_drug[index][index_2]=X_drug_drug_index/(sum(X_drug_drug[index])+sum(X_drug_drug[index_2])-X_drug_drug_index)
                    Sim_drug_drug[index_2][index]=X_drug_drug_index/(sum(X_drug_drug[index])+sum(X_drug_drug[index_2])-X_drug_drug_index)

def sim_feature(name,size):
    drug_sim_feature=np.zeros((size,size))
    with open(name,'r') as r:
        line_index=0
        for line in r:
            line=line.strip('\n').split("\t")
            for index,value in enumerate(line):
                if line_index in drug_tran and index in drug_tran:
                    drug_sim_feature[drug_tran[line_index]][drug_tran[index]]=float(value)               
            line_index+=1
    return drug_sim_feature
Sim_drug_disease = sim_feature('Sim_mat_drug_disease.txt', drug_size)
#Sim_drug_drug = sim_feature('Sim_mat_drug_drug.txt', drug_size)    

# Feature SVD

In [41]:
Svd_dim = 30

u_P, s_P, vh_P = np.linalg.svd(X_off, full_matrices=True)
X_off_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(X_tar, full_matrices=True)
X_tar_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(X_struct, full_matrices=True)
X_struct_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(X_atc, full_matrices=True)
X_atc_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(X_drug_disease, full_matrices=True)
X_drug_disease_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(X_drug_drug, full_matrices=True)
X_drug_drug_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
X_original_svd = np.hstack((X_off_svd, X_tar_svd))
X_original_svd = np.hstack((X_original_svd, X_struct_svd))
X_original_svd = np.hstack((X_original_svd, X_atc_svd))
X_original_svd = np.hstack((X_original_svd, X_drug_disease_svd))
X_original_svd = np.hstack((X_original_svd, X_drug_drug_svd))
X_original_svd = torch.from_numpy(X_original_svd).to("cpu").cuda()

u_P, s_P, vh_P = np.linalg.svd(Sim_drug_disease, full_matrices=True)
Sim_drug_disease_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(Sim_drug_drug, full_matrices=True)
Sim_drug_drug_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(Sim_sider, full_matrices=True)
Sim_sider_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(Sim_target, full_matrices=True)
Sim_target_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(Sim_atc, full_matrices=True)
Sim_atc_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
u_P, s_P, vh_P = np.linalg.svd(Sim_struct, full_matrices=True)
Sim_struct_svd = u_P[:,0:Svd_dim].dot(np.sqrt(np.sqrt(np.diag(s_P[:Svd_dim]))))
X_similarity_svd = np.hstack((Sim_sider_svd, Sim_target_svd))
X_similarity_svd = np.hstack((X_similarity_svd, Sim_struct_svd))
X_similarity_svd = np.hstack((X_similarity_svd, Sim_atc_svd))
X_similarity_svd = np.hstack((X_similarity_svd, Sim_drug_disease_svd))
X_similarity_svd = np.hstack((X_similarity_svd, Sim_drug_drug_svd))
X_similarity_svd = torch.from_numpy(X_similarity_svd).to("cpu").cuda()


# Drug combinations

In [48]:
dtype = torch.float
device = torch.device("cuda:0") 
y_true=np.zeros((drug_size,drug_size))
drug=[]
Gold_dict={}
with open("gold_combination.txt",'r') as f:
        for i in f:
            i=i.strip('\n').split('\t')
            if i[0]!=i[1]:
                Gold_dict[i[0]+'_'+i[1]]=1
                Gold_dict[i[1]+'_'+i[0]]=1

with open("drug_name.txt") as r:
    for line in r:
        line=line.strip('\n')
        drug.append(line)

# Model

In [58]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from scipy import sparse
from utils import SparseGraphConvolution

class GCNEncoderWithFeatures(nn.Module):
    def __init__(self, num_features: int,
                 dropout: float = 0.3, bias: bool = False):
        super(GCNEncoderWithFeatures, self).__init__()
        self.input_dim = num_features
        self.dropout = nn.Dropout(dropout)
        self.bias = bias
        gcn_hidden1=180
        gcn_hidden2=180
        GC = SparseGraphConvolution 
        self.gc_input = GC(in_features = num_features, out_features = gcn_hidden1, bias = bias)
        self.gc_hidden1 = GC(in_features = gcn_hidden1, out_features = gcn_hidden2, bias = bias)
        self.trans_h = nn.Linear(gcn_hidden1 + num_features, gcn_hidden1, bias = bias)
        self.trans_h1 = nn.Linear(gcn_hidden2 + num_features, gcn_hidden2, bias = bias)
    def forward(self, features: torch.Tensor, adj: torch.sparse.FloatTensor) -> torch.Tensor:
        hidden1 = F.relu(self.trans_h(torch.cat([self.gc_input(features, adj), features], dim=1)))
        hidden1 = self.dropout(hidden1)
        hidden2 = F.relu(self.trans_h1(torch.cat([self.gc_hidden1(hidden1, adj), features], dim=1)))
        return hidden1,hidden2

class HierGlobalGCN(nn.Module):
    def __init__(self, num_features: int,
                 dropout: float = 0.3):
        super(HierGlobalGCN, self).__init__()
        self.num_features = num_features
        self.dropout = nn.Dropout(dropout)
        self.global_enc = GCNEncoderWithFeatures(num_features)

    def forward(self,
               Ori, Sim, Adj, return_embeddings: bool = False):
        Ori = self.dropout(Ori)
        Ori_h1, Ori_h2 = self.global_enc(Ori.to(torch.float32).cuda(), Adj.to(torch.float32).cuda())
        Ori_combination = torch.cat((Ori_h1, Ori_h2),1)
        Ori_combination = self.dropout(Ori_combination)  

        Sim = self.dropout(Sim)
        Sim_h1, Sim_h2 = self.global_enc(Sim.to(torch.float32).cuda(), Adj.to(torch.float32).cuda())
        Sim_combination = torch.cat((Sim_h1, Sim_h2),1)
        Sim_combination = self.dropout(Sim_combination)  
        Fin_fea = torch.cat((Ori_combination,Sim_combination),1)
        output = Fin_fea.mm(Fin_fea.T)
        return  output


# Run

In [73]:
import random
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn import preprocessing  
from sklearn.metrics import auc
from scipy.stats import pearsonr,spearmanr
import scipy.sparse as sp
from torch.optim import SGD
min_max_scaler = preprocessing.MinMaxScaler() 
loss_MSE=torch.nn.MSELoss(reduction='none')
def normalize_adj(adj: sp.csr_matrix) -> sp.coo_matrix:
    adj = sp.coo_matrix(adj)
    adj_ = adj
    rowsum = np.array(adj_.sum(0))
    rowsum_power = []
    for i in rowsum:
        for j in i:
            if j !=0 :
                j_power = np.power(j, -0.5)
                rowsum_power.append(j_power)
            else:
                j_power = 0
                rowsum_power.append(j_power)
    rowsum_power = np.array(rowsum_power)
    degree_mat_inv_sqrt = sp.diags(rowsum_power)
    adj_norm = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    return adj_norm

def sparse_mx_to_torch_sparse_tensor(sparse_mx) \
        -> torch.sparse.FloatTensor:
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64)
    )
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


High_neighbour_number = 5
aucss,aupr,recall,f1=[],[],[],[]
for cross_index in range(10):
    dicts_train_y_true,dicts_test_y_true,dict_test_list,dict_train_list,dict_train_list_only_p={},{},{},{},{}
    for index in range(5):
        if index not in dicts_train_y_true:
            dicts_train_y_true[index]=np.zeros((len(drug),len(drug)))
            dicts_test_y_true[index]=np.zeros((len(drug),len(drug)))
            dict_test_list[index]=[]
            dict_train_list[index]=[]
            dict_train_list_only_p[index]=[]  #gold drug combination position in train set
    for index_out,value_out in enumerate(drug):
        for index_inn,value_in in enumerate(drug):
            if index_inn > index_out:
                temp = random.randint(0,4)
                if value_out+'_'+value_in in Gold_dict :
                    dicts_test_y_true[temp][index_out][index_inn]=1
                    dict_test_list[temp].append((index_out,index_inn))
                    dicts_test_y_true[temp][index_inn][index_out]=1
                    dict_test_list[temp].append((index_inn,index_out))
                else:
                    #  if random.randint(0,29)==1:
                    dict_test_list[temp].append((index_out,index_inn))        
                    dict_test_list[temp].append((index_inn,index_out))
                for index in range(5):
                    if temp!=index:
                        if value_out+'_'+value_in in Gold_dict :
                            dicts_train_y_true[index][index_out][index_inn]=1
                            dict_train_list[index].append((index_out,index_inn))
                            dicts_train_y_true[index][index_inn][index_out]=1
                            dict_train_list[index].append((index_inn,index_out))   
                            dict_train_list_only_p[index].append((index_out,index_inn))
                            dict_train_list_only_p[index].append((index_inn,index_out))             
                        else:
                        #    if random.randint(0,29)==1:
                            dict_train_list[index].append((index_out,index_inn))
                            dict_train_list[index].append((index_inn,index_out))
    for key in dicts_train_y_true.keys():            
        dict_train_list[key]=np.array(dict_train_list[key])                 
        dict_test_list[key]=np.array(dict_test_list[key])
        dicts_train_y_true[key]=torch.from_numpy(dicts_train_y_true[key]).to(device)
        dicts_test_y_true[key]=torch.from_numpy(dicts_test_y_true[key]).to(device)

    for inde in range(5):
        model = HierGlobalGCN(num_features=180).to(device)
        dicts_test={}
        for i in range(len(dict_test_list[inde])):
            dicts_test[str(dict_test_list[inde][i][0])+'_'+str(dict_test_list[inde][i][1])]=1
        mask = dicts_train_y_true[inde].clone()
        Weight = len(dict_train_list[inde])/sum(sum(dicts_train_y_true[inde]))
        dicts_train={}
        for i in range(len(dict_train_list[inde])):
            dicts_train[str(dict_train_list[inde][i][0])+'_'+str(dict_train_list[inde][i][1])]=1
        for i in range(drug_size):
            for j in range(drug_size):
                if str(i)+'_'+str(j)  not in dicts_test:
                    if mask[i][j]==1:
                        mask[i][j]=Weight/2#math.log2(Weight)
                    elif str(i)+'_'+str(j) in dicts_train :
                        mask[i][j]=1


        optimizer = SGD(params=model.parameters(), lr=1e-6, momentum=0.9)
        '''
        High_neighbour
        '''
        combine_svd = torch.cat((X_original_svd,X_similarity_svd),dim=1)
        combine_svd = combine_svd.to("cpu").detach().numpy()
        Sim_combine_svd = np.zeros((drug_size, drug_size))
        for index_tem in range(drug_size):
            for index_2_tem in range(drug_size):
                Sim_combine_svd[index_tem][index_2_tem] = spearmanr(combine_svd[index_tem],combine_svd[index_2_tem])[0]
        Sim_combine_svd = torch.from_numpy(Sim_combine_svd).to("cpu").cuda()
        Sim_combine_svd = Sim_combine_svd - torch.eye(drug_size).cuda()
        fin_com_sim = torch.zeros((drug_size, drug_size))
        for index_eye in range(drug_size):
                fin_com_sim[index_eye]=(Sim_combine_svd[index_eye]-torch.min(torch.topk(Sim_combine_svd, High_neighbour_number, dim=1).values,dim=1).values[index_eye])
        
        new_dict_train_list_only_p=[]
        for value in dict_train_list_only_p[inde]:
            new_dict_train_list_only_p.append(value)
        for inde_1 in range(drug_size):
                for inde_2 in range(drug_size):
                    if inde_2>inde_1 and fin_com_sim[inde_1][inde_2]>=0:
                        new_dict_train_list_only_p.append((inde_2,inde_1))
                        new_dict_train_list_only_p.append((inde_1,inde_2))
        data_train= np.ones(len(new_dict_train_list_only_p))
        adj = sp.csr_matrix((data_train, (np.array(new_dict_train_list_only_p)[:, 0], np.array(new_dict_train_list_only_p)[:, 1])),              
                                shape=(drug_size, drug_size))   
                                

        adj_norm = normalize_adj(adj)         
        adj_norm = sparse_mx_to_torch_sparse_tensor(adj_norm).cuda()
        maxs,aucs,auprs,flag=0,0,0,0
        for index in range(1000):

            model.train()
            optimizer.zero_grad()
            y_pred = model(X_original_svd.to(torch.float32),X_similarity_svd.to(torch.float32).detach(),adj_norm.to(torch.float32))

            dicts_train_y_true[inde]=dicts_train_y_true[inde]

            loss = torch.sum(loss_MSE(y_pred.to("cpu").to(torch.float32), dicts_train_y_true[inde].to("cpu").to(torch.float32))* mask.to("cpu"))
            loss.to(torch.float32).backward()

            optimizer.step()
            if index %10 == 0:
                precision, recall, pr_thresholds = precision_recall_curve(dicts_train_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_train_list[inde][:,0]),list(dict_train_list[inde][:,1])],y_pred.to("cpu",torch.double).detach().numpy()[list(dict_train_list[inde][:,0]),list(dict_train_list[inde][:,1])], pos_label=1)
                aupr_score = auc(recall, precision) 
                fpr, tpr, thresholds = roc_curve(dicts_train_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_train_list[inde][:,0]),list(dict_train_list[inde][:,1])], y_pred.to("cpu",torch.double).detach().numpy()[list(dict_train_list[inde][:,0]),list(dict_train_list[inde][:,1])], pos_label=1)           
                if np.trapz(tpr,fpr) > maxs:
                    flag=1
                    maxs=np.trapz(tpr,fpr)
                precision, recall, pr_thresholds = precision_recall_curve(dicts_test_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])],y_pred.to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])], pos_label=1)
                aupr_score = auc(recall, precision) 
                fpr, tpr, thresholds = roc_curve(dicts_test_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])], y_pred.to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])], pos_label=1)
                if flag==1:
                    flag=0
                    torch.save(model, "../model/DCGCN_model") 
        '''
        DCGCN test
        '''
        model = torch.load("../model/DCGCN_model")
        model.eval()
        y_pred = model(X_original_svd.to(torch.float32),X_similarity_svd.to(torch.float32).detach(),adj_norm.to(torch.float32))

        precision, recall, pr_thresholds = precision_recall_curve(dicts_train_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_train_list[inde][:,0]),list(dict_train_list[inde][:,1])],y_pred.to("cpu",torch.double).detach().numpy()[list(dict_train_list[inde][:,0]),list(dict_train_list[inde][:,1])], pos_label=1)
        aupr_score = auc(recall, precision) 
        fpr, tpr, thresholds = roc_curve(dicts_train_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_train_list[inde][:,0]),list(dict_train_list[inde][:,1])], y_pred.to("cpu",torch.double).detach().numpy()[list(dict_train_list[inde][:,0]),list(dict_train_list[inde][:,1])], pos_label=1)
        precision, recall, pr_thresholds = precision_recall_curve(dicts_test_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])],y_pred.to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])], pos_label=1)
        aupr_score = auc(recall, precision) 
        print(aupr_score)
        aupr.append(aupr_score)
        fpr, tpr, thresholds = roc_curve(dicts_test_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])], y_pred.to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])], pos_label=1)
        print(str(np.trapz(tpr,fpr)))  
        aucss.append(np.trapz(tpr,fpr))
        y_pred_minMax = min_max_scaler.fit_transform(y_pred.to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])].reshape(-1, 1))
        y_pred_minMax[y_pred_minMax>0.5]=1
        y_pred_minMax[y_pred_minMax<=0.5]=0
        print(f1_score(dicts_test_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])],y_pred_minMax))
        f1.append(f1_score(dicts_test_y_true[inde].to("cpu",torch.double).detach().numpy()[list(dict_test_list[inde][:,0]),list(dict_test_list[inde][:,1])],y_pred_minMax))
        print("22---------------epoch------------------22")


0.2598173812279342
0.9624333939196221
0.3673469387755102
22---------------epoch------------------22
0.39788780354491055
0.9435723090727729
0.4574468085106383
22---------------epoch------------------22
0.2721607709006746
0.9308813889822245
0.3365384615384615
22---------------epoch------------------22
0.25521293409149987
0.9450299456799294
0.3387978142076503
22---------------epoch------------------22
0.3074865597185732
0.952629207927872
0.32307692307692304
22---------------epoch------------------22
0.32205212785876364
0.9336729645785475
0.37499999999999994
22---------------epoch------------------22
0.2780361908326095
0.960322794700002
0.3350785340314136
22---------------epoch------------------22
0.3119300115299478
0.9517605001257501
0.41463414634146334
22---------------epoch------------------22
0.36132433590838253
0.9331478487652447
0.18644067796610167
22---------------epoch------------------22
0.24579436736191101
0.9447766127193983
0.3014705882352941
22---------------epoch--------------

In [74]:
print(np.mean(aucss)) 
print(np.mean(aupr))
print(np.mean(f1))
print(np.std(aucss)) 
print(np.std(aupr))
print(np.std(f1))

0.9445328320620212
0.2965332529017064
0.34391344420895875
0.013097623992782238
0.052534480238029556
0.07176721413212694
